In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Noriu paanalizuot, kokią įtaką specialybių automatizavimas turės visai JAV ir atskiroms valstijoms atskirai.


Specialybių automatizavimo Duomenu importavimas

In [ ]:
Automation_data = "../input/occupation-salary-and-likelihood-of-automation/automation_data_by_state.csv"
A_data = pd.read_csv(Automation_data, encoding = "ISO-8859-1")
state_names = A_data.columns[3:]
A_data.head()
# A_data.isnull().sum() - patikrinau, kad stulpeliuose nėra null verčių.

Specialybiu uzdarbio Duomenu importavimas

In [ ]:
salary_data = "../input/occupation-salary-and-likelihood-of-automation/occupation_salary.xlsx"
S_data = pd.read_excel(salary_data, index_col="OCC_CODE")
# S_data.shape
# S_data.isnull().sum() # tik ANNUAL, HOURLY stulpeliuose daug null reikšmių. Kituose stulpeliuose nėra.

Drop emty columns: ANNUAL, HOURLY

In [ ]:
S_data_clean = S_data.drop(['ANNUAL', 'HOURLY'], axis=1)
S_data_clean

Kad palyginti specialybiu specialistų skaičių visoj populiacijoj, pasiėmiau JAV populiacijų duomenis iš wiki

I couldn't read html in the Kaggle platorm, so I made csv data file in spider. The code in spyder here:

In [ ]:
# link = 'https://www.infoplease.com/us/states/state-population-by-rank'
# # w = pd.read_html(link, header=0)
# w[0].columns
# df_1 = w[0]
# condition = df_1['State'].isin(state_names)
# population_0 = df_1[condition]
# population.head() #- Census population - valstijos visos pupuliacijos skaičiai.
# null verčių stulpeliuose nėra

 Then I opened the US_population csv here:

In [ ]:
link = '../input/us-population-salary/US_population_2.csv'
population = pd.read_csv(link, header=0)
population.head() #- Census population - valstijos visos pupuliacijos skaičiai.
# null verčių stulpeliuose nėra

# Duomenų tvarkymas

Dar pries rusiavima, susumuoju valstijų duomenis, kad turėt visos US duomenis, susirast Occupations index names su 0 darbuotojų ir panaikint eilutes pagrindiniuose duomenyse. I sort occupation by US_workers

In [ ]:
us_sum = A_data[state_names].sum(axis=1)
us_sum_DF = pd.DataFrame({'US_workers':us_sum.values})
Occupation_proba = A_data[['Occupation', 'Probability']]
US_O_proba = Occupation_proba.join(us_sum_DF)
index_names = US_O_proba[ US_O_proba['US_workers'] == 0 ].index
US_worker = US_O_proba.drop(index_names).reset_index()
US_work = US_worker.sort_values(by=['US_workers'], ascending=False)
US_work

Take 100 most common accupation in US

In [ ]:
US100 = US_work.head(100)
US100

# Drop zero lines from A_data

In [ ]:
A_data_clean = A_data.drop(index_names).reset_index()
# A_data_clean
# A_data.iloc[index_names] # - droped lines rom A_data
A_data_SOC = A_data_clean.set_index('SOC')

# English Transformuot valstiju darbuotoju letele i skaicius dalis nuo populiacijos. didžiausi/mažiausi specialybiu skaiciai pagal valstijas (sns.swarmplot - Categorical scatter plots )

# A_data transformavimas

In [ ]:
col_names = A_data_clean.columns
A_trans = pd.DataFrame(A_data_clean.values.T, columns=A_data_clean['SOC'], index=col_names)
A_tr = A_trans.iloc[4:]

In [ ]:
population_sort = population.sort_values(by=['State'])

In [ ]:
states_pop = population_sort.loc[:, ['State', 'July 2019 Estimate']].reset_index()
states_pop.head()

Number of occupations jobs in states divided by population of state. Then the data frame transformed back

In [ ]:
reliative_popul = A_tr.div(states_pop['July 2019 Estimate'].values,axis=0)
reliative_popul = reliative_popul.fillna(0)
# heat_map = reliative_popul.iloc[:, np.r_[0:50]]
A_double_T = reliative_popul.T

Join occupation and probability coulumns back to transformed data frame.

In [ ]:
A_data_double_T = A_data_SOC[['Occupation', 'Probability']].join(A_double_T)
A_data_double_T.head()

Split SOC culumn to group by occupations into categories.

In [ ]:
SOC_column = A_data_double_T.reset_index()
SOC_column.head()

In [ ]:
SOC_column['SOC'] = SOC_column.SOC.astype(str)

In [ ]:
SOC_column[['Occupation_group_no','Occupation_no']] = SOC_column.SOC.str.split("-",expand=True,)
Group_data = SOC_column.copy()
Group_data.head()

Occupations groups probabilities for automation

In [ ]:
Group = Group_data.groupby(['Occupation_group_no']).Probability.mean()
Group

Occupations groups ratio to population per state

In [ ]:
Group_states = Group_data.groupby(['Occupation_group_no'])[state_names].sum()
Group_states

Titles of occupations groups

In [ ]:
s = pd.Series(['Management', 'Business Operations', 'Computer and Mathematical', 'Architecture and Engineering', 'Life, Physical, and Social Science', 'Community and Social Service', 'Legal', 'Education, Training, and Library', 'Design, Entertainment and Sports', 'Healthcare Practitioners', 'Healthcare Support', 'Protective Service', 'Food Serving Related', 'Cleaning and Maintenance', 'Personal Care and Service', 'Sales and Related', 'Administrative Support', 'Farming, Fishing, and Forestry', 'Construction and Extraction', 'Installation and Repair', 'Production', 'Transportation'], index=['11', '13', '15', '17', '19', '21', '23', '25', '27', '29', '31', '33', '35', '37', '39', '41', '43', '45', '47', '49', '51', '53'])
s.head()

Make data frame from series

In [ ]:
Occupations_groups_pd = pd.DataFrame({'index':s.index, 'Occupations groups':s.values})
Occupations_groups = Occupations_groups_pd.set_index('index')

Join data to one table and sort by probability

In [ ]:
Occupations_groups_join1 = Occupations_groups.join(Group)
Occupations_groups_join2 = Occupations_groups_join1.join(Group_states)
Occupations_groups_prob_sort1 = Occupations_groups_join2.sort_values(by=['Probability'], ascending=False).reset_index()
Occupations_groups_prob_sort = Occupations_groups_prob_sort1.fillna(0)
Occupations_groups_prob_sort.head()

Merge Occupations groups and probability columns to one

In [ ]:
Occupations_groups_prob_round = Occupations_groups_prob_sort['Probability'].round(2)
Occupations_groups_prob_round_df = pd.DataFrame({'Probability':Occupations_groups_prob_round.values})

In [ ]:
Occupations_groups_prob_sort["Occupations groups and Probability"] = Occupations_groups_prob_sort["Occupations groups"] + " " + Occupations_groups_prob_round_df["Probability"].astype(str)
Occupations_groups_plot = Occupations_groups_prob_sort.copy().set_index('Occupations groups and Probability')
Occupations_groups_plot

Plot heat map

In [ ]:
sns.set_style("dark")
plt.figure(figsize=(30,20))
# plt.title("Occupations by US states")
sns.heatmap(data=Occupations_groups_plot[state_names]) 
# plt.xlabel("States")
plt.ylabel("")
sns.set(font_scale=4)
plt.savefig('All states Occupations groups and probability of automatisation.svg', bbox_inches="tight")
plt.show() 

In [ ]:
sns.set_style("dark")
plt.figure(figsize=(30,20))
# plt.title("Few states Occupations groups and probability of automatisation")
sns.set(font_scale=3)
sns.heatmap(data=Occupations_groups_plot[['South Dakota', 'Nevada', 'District of Columbia', 'Massachusetts']]) 
fig = plt.ylabel("")
plt.savefig('Few states Occupations groups and probability of automatisation.svg', bbox_inches="tight")
plt.show() 

In [ ]:
# fig.savefig("Sensitive States occupations groups.jpg")
# figure.savefig('Sensitive States occupations groups.png')

Safest and dangerous accupations for automatisation

In [ ]:
A_data_clean.head()

In [ ]:
A_data_prob_sort = A_data_clean.sort_values(by=['Probability'], ascending=False).reset_index()
A_head_clean = A_data_prob_sort.head()
A_tail_clean = A_data_prob_sort.tail()

A_data_highest_lowest_prob = pd.concat([A_head_clean, A_tail_clean])
A_data_highest_lowest_prob_present = A_data_highest_lowest_prob.drop(['level_0', 'index'], axis=1).set_index('Occupation')
A_data_highest_lowest_prob_present

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(30,20))
# plt.title("Occupations by US states")
sns.heatmap(data=A_data_highest_lowest_prob_present[state_names]) 
# plt.xlabel("States")
plt.ylabel("")
sns.set(font_scale=4)
plt.savefig('The safest and the most dangerous occupations for automatisation.svg', bbox_inches="tight")
plt.show() 

# Heat map

Persusiuot valstijas pagal populiacijas. Jujunt i viena stulpeli States ir ju procenta nuo US populiacijos. Dropint specelybes, kuriu US nera - ir visur naudot tik tuos duomenis.
Sujunt i viena stulpeli specalybes ir ju tikimybes automatizavimo. Brezt heat map. Gal reikes atmest spec su vidutinem tikimybem ar pan

# Plot Sumuotus visu valstiju gyventojus ir ziuret kokios specialybės sudaro didžiasią/mažiausią dali gyventoju (sort) histogram? - scater graphs (regretion relationship) populiacijos dalies nuo automatizavimo tikimybės.

In [ ]:
common_US_work = US_work.reset_index().drop(['level_0', 'index'], axis=1)

In [ ]:
sns.regplot(y=US_work['US_workers'], x=US_work['Probability'])

US occupations number ant accupations probabilities distribution

In [ ]:
plt.figure(figsize=(15,10))
# sns.distplot(a=US_work['US_workers'], kde=False)
sns.barplot(x=common_US_work.index, y=common_US_work['US_workers'])
plt.title("US occupations distribution")
# Add label for horizontal axis
plt.xlabel("Occupation")

Check, what is propabilities distribution

In [ ]:
US_work

# Occupations with highest anf lowest automation probabilities

In [ ]:
A_prob_sort = US_work.sort_values(by=['Probability'], ascending=False).reset_index()
A_head = A_prob_sort.head()
A_tail = A_prob_sort.tail()

highest_lowest_prob = pd.concat([A_head, A_tail])
highest_lowest_prob_present = highest_lowest_prob.drop(['level_0', 'index'], axis=1)
highest_lowest_prob_present

# US workers numbers and Occupations automatisation probabilities distribution

In [ ]:
# plt.title("US workers numbers and Occupations automatisation probabilities distribution")
sns.set_style("dark")
plt.figure(figsize=(20,20))
fig = sns.jointplot(x=US_work['Probability'], y=US_work['US_workers'], kind="kde")
# plt.ylabel("Occupations")
sns.set(font_scale=1.2)
# ax.set_ylim(1,31)
# ax.set_yticks(range(1,32))
# g.despine(bottom=True, left=True)
plt.gcf().set_size_inches(12, 8)
plt.show() 

Išsaugau grafiką, kaip svg faila.

In [ ]:
# fig = sns.jointplot(x=US_work['Probability'], y=US_work['US_workers'], kind="kde")
# fig.savefig("output.svg")

In [ ]:
US_sort_probability = US_work.sort_values(by=['Probability'], ascending=False).reset_index().drop(['level_0', 'index'], axis=1)
# US_sort_probability

Probability higher than 0.7  representing a "high risk category, meaning that associated occupations are potentially automatable over some unspecified number of years, perhaps a decade or two" according to the original research paper. We can look to this probability as to a time frame, where higher propabilty occupations are likely to be automated sooner.

The lost work positions and state population ratio. I take lost work position, when work automation probability ir equal to 0.7 or higher (threshhold >= 0.7).

In [ ]:
threshhold = 0.7 # accupations automation probability threshhold

# Total number of jobs positions per state

I estimated, that these total job numbers are about 10%, due to jabs position not included. I got data with all jobs where data was not available or there were less than 10 employees were marked as zero.

In [ ]:
p = A_data_clean.sort_values(by=['Probability'], ascending=False)
sum_work_per_state = p[state_names].sum()
States_sum_DF = pd.DataFrame({'States':sum_work_per_state.index, 'sum_work_positions':sum_work_per_state.values})
States_sum_sort =  States_sum_DF.sort_values(by=['sum_work_positions'], ascending=False)

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(y=States_sum_sort['States'], x=States_sum_sort['sum_work_positions'])
plt.title("Total number of jobs positions per state")
plt.xlabel("Number of jobs positions")

# Plot second bar to every state - left absolute occupation positions when threshhold < 0.7.

In [ ]:
p_index = p.reset_index().drop(['level_0', 'index', 'SOC'], axis=1)
p05 = p_index.loc[(p_index.Probability < threshhold)]

In [ ]:
sum_work_per_state05 = p05[state_names].sum()
States_sum_DF05 = pd.DataFrame({'States':sum_work_per_state05.index, 'sum_work_positions':sum_work_per_state05.values})
States_sum_sort05 =  States_sum_DF05.sort_values(by=['sum_work_positions'], ascending=False)
States_sum_sort05.head()

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(y=States_sum_sort05['States'], x=States_sum_sort05['sum_work_positions'])
plt.title("Total number of jobs positions per state")
plt.xlabel("Number of jobs positions")

# How many jobs positions would be lost in States, if we lost accupations which have automation probability equal to 0.7 or higher.

In [ ]:
States_sum_DF['Threshhold'] = 1.0
States_sum_DF05['Threshhold'] = threshhold
States_sum_DF05.head()

In [ ]:
Compare_sums = pd.concat([States_sum_DF, States_sum_DF05])
Compare_sums_sort = Compare_sums.sort_values(by=['sum_work_positions'], ascending=False)
Compare_sums_sort

In [ ]:
plt.figure(figsize=(18,13))
sns.set_style("darkgrid")
sns.barplot(y=Compare_sums_sort['States'], x=Compare_sums_sort['sum_work_positions'], hue='Threshhold', data=Compare_sums_sort)#, kind='bar')
sns.set(font_scale=1.5)
# plt.title("Total number of jobs positions per state now (Threshhold=1.0) and when higher automatisation probability accuaptions lost (Threshhold=threshhold)")
plt.xlabel("Number of jobs positions")
plt.savefig('Probabilities 1 and 07.svg', bbox_inches="tight")
plt.show()

Lets look, what are reliative loss numbers

In [ ]:
States_sum_join = States_sum_DF.join(States_sum_DF05, lsuffix='1.0', rsuffix='0.5')
States_sum_drop = States_sum_join.drop(['Threshhold1.0', 'States0.5', 'Threshhold0.5'], axis=1)
Relative_jobs_drop = ((States_sum_drop['sum_work_positions1.0']-States_sum_drop['sum_work_positions0.5'])/States_sum_drop['sum_work_positions1.0'])*100
States_sum_drop.head()

In [ ]:
Relative_jobs_drop_DF = pd.DataFrame({'Lost jobs ratio':Relative_jobs_drop.values})
Relative_jobs_drop_States = States_sum_drop.join(Relative_jobs_drop_DF)
Relative_jobs_drop_States_sort = Relative_jobs_drop_States.sort_values(by=['sum_work_positions1.0'], ascending=False)
Relative_jobs_drop_mean = Relative_jobs_drop_States_sort['Lost jobs ratio'].mean()
Relative_jobs_drop_mean

In [ ]:
sns.set_palette("cubehelix")
plt.figure(figsize=(18,13))
graph = sns.barplot(y=Relative_jobs_drop_States_sort['States1.0'], x=Relative_jobs_drop_States_sort['Lost jobs ratio'])#, hue='Threshhold', data=Compare_sums_sort)#, kind='bar')
# plt.title("Lost jobs ratio per state when we lost jobs with automatisation probability equal to 0.5 (Threshhold) or higher")
plt.xlabel("Lost jobs ratio")
plt.ylabel("States")
graph.axvline(Relative_jobs_drop_mean)
plt.savefig('Lost jobs ratio per state.svg', bbox_inches="tight")
plt.show()

In [ ]:
Relative_jobs_drop_States_highest = Relative_jobs_drop_States.sort_values(by=['Lost jobs ratio'], ascending=False)

In [ ]:
sns.set_palette("cubehelix")
plt.figure(figsize=(18,12))
graph = sns.barplot(y=Relative_jobs_drop_States_highest['States1.0'], x=Relative_jobs_drop_States_highest['Lost jobs ratio'])#, hue='Threshhold', data=Compare_sums_sort)#, kind='bar')
# plt.title("Lost jobs ratio per state when we lost jobs with automatisation probability equal to 0.5 (Threshhold) or higher")
plt.xlabel("Lost jobs ratio")
plt.ylabel("States")
graph.axvline(Relative_jobs_drop_mean)
plt.savefig('Lost jobs ratio sorted.svg', bbox_inches="tight")
plt.show()

# South Dakota and Nevada have most jobs losses. Lets look, what are biggest occupations they lost

Let's start with Nevada data

In [ ]:
nevada = A_data_clean[['Occupation', 'Probability', 'South Dakota', 'Nevada']].sort_values(by=['Probability'], ascending=False)
nevada07_full = nevada.loc[(nevada.Probability >= threshhold)].sort_values(by=['Nevada'], ascending=False).reset_index()
nevada07 = nevada07_full.head(9)
nevada07_tail = nevada07_full.tail(308).Nevada.sum()
df2 = {'Occupation': 'Other', 'Probability': 0, 'South Dakota': 0, 'Nevada': nevada07_tail} 
nevada07 = nevada07.append(df2, ignore_index = True) 
nevada07

Plot pie chart of Nevada data

In [ ]:
# Create a list of colors (from iWantHue)
colors = ["#E13F29", "#D69A80", "#D63B59", "#AE5552", "#CB5C3B", "#EB8076", "#96624E"]

# Create a pie chart
plt.pie(
    # using data total)arrests
    nevada07['Nevada'],
    # with the labels being officer names
    labels=nevada07['Occupation'],
    # with no shadows
    shadow=False,
    # with colors
    colors=colors,
    # with one slide exploded out
    explode=(0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    # with the start angle at 90%
    startangle=90,
    # with the percent listed as a fraction
    autopct='%1.1f%%',
    )
# plt.title(label="Nevada states the bigest occuaptions", fontsize=20, loc="left", color="red")
sns.set(font_scale=1.3)
# View the plot drop above
plt.axis('equal')

# View the plot
plt.tight_layout()
plt.savefig('Nevada pie.svg', bbox_inches="tight")
plt.show()

Now let's look to South Dakota

In [ ]:
dakota = A_data_clean[['Occupation', 'Probability', 'South Dakota']].sort_values(by=['Probability'], ascending=False)
dakota07_full = dakota.loc[(dakota.Probability >= threshhold)].sort_values(by=['South Dakota'], ascending=False).reset_index()
dakota07 = dakota07_full.head(9)
dakota07_tail = dakota07_full.tail(308)['South Dakota'].sum()
df2 = {'Occupation': 'Other', 'Probability': 0, 'South Dakota': dakota07_tail} 
dakota07 = dakota07.append(df2, ignore_index = True) 
dakota07

Plot pie chart of South Dakota data

In [ ]:
# Create a pie chart
plt.pie(
    # using data total)arrests
    dakota07['South Dakota'],
    # with the labels being officer names
    labels=dakota07['Occupation'],
    # with no shadows
    shadow=False,
    # with colors
    colors=colors,
    # with one slide exploded out
    explode=(0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    # with the start angle at 90%
    startangle=90,
    # with the percent listed as a fraction
    autopct='%1.1f%%',
    )
# plt.title(label="Nevada states the bigest occuaptions", fontsize=20, loc="left", color="red")
sns.set(font_scale=1.3)
# View the plot drop above
plt.axis('equal')

# View the plot
plt.tight_layout()
plt.savefig('Dakota pie.svg', bbox_inches="tight")
plt.show()

District of Columbia has lowest sensitivity for automation. Let's check the data

In [ ]:
DC = A_data_clean[['Occupation', 'Probability', 'District of Columbia']].sort_values(by=['Probability'], ascending=False)
DC07_full = DC.loc[(DC.Probability >= threshhold)].sort_values(by=['District of Columbia'], ascending=False).reset_index()
DC07 = DC07_full.head(9)
DC07_tail = DC07_full.tail(308)['District of Columbia'].sum()
df2 = {'Occupation': 'Other', 'Probability': 0, 'District of Columbia': DC07_tail} 
DC07 = DC07.append(df2, ignore_index = True) 
DC07

District of Columbia pie chart

In [ ]:
# Create a pie chart
plt.pie(
    # using data total)arrests
    DC07['District of Columbia'],
    # with the labels being officer names
    labels=DC07['Occupation'],
    # with no shadows
    shadow=False,
    # with colors
    colors=colors,
    # with one slide exploded out
    explode=(0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    # with the start angle at 90%
    startangle=90,
    # with the percent listed as a fraction
    autopct='%1.1f%%',
    )
# plt.title(label="Nevada states the bigest occuaptions", fontsize=20, loc="left", color="red")
sns.set(font_scale=1.3)
# View the plot drop above
plt.axis('equal')

# View the plot
plt.tight_layout()
plt.savefig('District of Columbia pie.svg', bbox_inches="tight")
plt.show()

In [ ]:
A_data_clean.head()

Massachusetts data

In [ ]:
massachusetts = A_data_clean[['Occupation', 'Probability', 'Massachusetts']].sort_values(by=['Probability'], ascending=False)
massachusetts07_full = massachusetts.loc[(nevada.Probability >= threshhold)].sort_values(by=['Massachusetts'], ascending=False).reset_index()
massachusetts07 = massachusetts07_full.head(9)
massachusetts07_tail = massachusetts07_full.tail(308).Massachusetts.sum()
df2 = {'Occupation': 'Other', 'Probability': 0, 'Massachusetts': massachusetts07_tail} 
massachusetts07 = massachusetts07.append(df2, ignore_index = True) 
massachusetts07

Massachusetts pie chart**

In [ ]:
# Create a pie chart
plt.pie(
    # using data total)arrests
    massachusetts07['Massachusetts'],
    # with the labels being officer names
    labels=massachusetts07['Occupation'],
    # with no shadows
    shadow=False,
    # with colors
    colors=colors,
    # with one slide exploded out
    explode=(0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    # with the start angle at 90%
    startangle=90,
    # with the percent listed as a fraction
    autopct='%1.1f%%',
    )
# plt.title(label="Nevada states the bigest occuaptions", fontsize=20, loc="left", color="red")
sns.set(font_scale=1.3)
# View the plot drop above
plt.axis('equal')

# View the plot
plt.tight_layout()
plt.savefig('Massachusetts pie.svg', bbox_inches="tight")
plt.show()

# Daugiausiai/mažiausiai uždirbanciu specialybiu išrušiavimas ir jų tikimybė būti automatizuotom.

Take only matching S_data occupations with A_data occupations 

In [ ]:
A_occupations = A_data_clean.SOC
condition = S_data_clean.index.isin(A_occupations)
Sdata_occupations = S_data_clean[condition]
Sdata_occupations.head()

Check if there are any zeros in Sdata_occupations column A_MEAN. There are no zeros.

# Workers income losses due to automation (threshhold 0.7)

In [ ]:
#SOC column set as index
A_data_SOC = A_data_clean.set_index('SOC')
A_data_SOC.head()

1. Join automation and salaries data tables. Agregate all occupations salaries per states

In [ ]:
all_data = A_data_SOC.join(Sdata_occupations)
all_data_A_mean = A_data_SOC.join(Sdata_occupations['A_MEAN'])
all_data_A_mean.head()

In [ ]:
State_jobs_money1 = all_data_A_mean[state_names].multiply(Sdata_occupations['A_MEAN'].values,axis=0).copy()
# All = State_jobs_money[state_names].div(10)
State_jobs_money = State_jobs_money1.fillna(0)
State_jobs_money

In [ ]:
State_jobs_money_per_states = State_jobs_money.sum(axis=0)
State_jobs_money_per_states_div_e6 = State_jobs_money_per_states.div(1000, axis=0,fill_value=0)
State_jobs_money_per_states



In [ ]:
States_sum_money_DF = pd.DataFrame({'States':State_jobs_money_per_state.index, 'State_jobs_money_per_state':State_jobs_money_per_state.values})
States_sum_money_sort =  States_sum_money_DF.sort_values(by=['State_jobs_money_per_state'], ascending=False)
States_sum_money_sort.head()

# How many occuaptions with probability >0.9 or >0.95?


# !!! Paimt 2 kartu transformuota A_data_clean DF su suskaiciuotais darbu santykiais

In [ ]:
probability_sort = A_data_clean.sort_values(by=['Probability'], ascending=False).reset_index()
probability50 = probability_sort.set_index('Occupation').head(50)
probability50

In [ ]:
plt.figure(figsize=(40,30))
plt.title("Occupations by US states")
sns.heatmap(data=probability50[state_names])#, annot=True) 
plt.xlabel("States")
plt.ylabel("Occupations")
sns.set(font_scale=5)
cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=30, as_cmap=True)
# sns.clustermap(data=corr, annot=True, fmt='d', cmap="Blues", annot_kws={"size": 16})

# Kokios specialybes turi didesne automatizavimo tikimybe nei 0.7?

# Kazkur idet boxplot (25, 50, 75), mean ...
# persentiles

# pasirinkti spalvu paletes grafikams
https://duckduckgo.com/?t=ffab&q=color+paletes&ia=web

Ziuret Tomo U ir kitų darbus, dėl ideju, ka dar paanlizuoti:
https://github.com/TK-Problem/Interesting_Sport_Stats

# How to mark numbers to every bar?

# jei liks laiko paprognozuot su ML. Python projektas, prie Gabrieles future.

Panagrinėti, kurie darbai turi didžiausią ir mažiausią tikimybę būti automatizuoti. Kurios valstijos gali prarasti didžiausią dalį darbo vietų/atlyginimų sumos? Korios mažiausiai paveikios automatizavimui(heat map).

Pažiūrėt ar šie valstijų duomenys koreliuoja su paskutiniais prezidento rinkimų duomenimis - ar valstijos balsavusios už Trumpą gali prarasti daugiausiai darbo vietų dėl automatizacijos.

In [ ]:
# # Angli6kai aprasyt projekta, nuo failu importavimo ir pan.
# # o skaidrese, tik pati tema. Bent 5 skaidrės tik su grafikais.

# # Heat map
# # Geoplot???

# # Paslept kodą, darbe.

# Perziurėt visą sintaksę. Ar suprantu, kas ką daro?

# openOffice turiu
# From Kotryna to Me:  (Privately) 02:58 PM
# gali but kad ir suopen office veiks
# tiesiog, kai padarysi savo apveiksliukus, pabandyk juos eksportavęs į SVG fail'ą įkelti į open office
# Kiek atsimnu, jis tas pačias funcijas turėtų turėti